In [1]:
!pip install python-sat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 10.4 MB/s eta 0:00:00


# Mục mới

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
from pysat.solvers import Glucose3, Solver
from prettytable import PrettyTable
from threading import Timer
import datetime
import pandas as pd
import os
from openpyxl import load_workbook
from openpyxl import Workbook
from zipfile import BadZipFile
from openpyxl.utils.dataframe import dataframe_to_rows
import time
from datetime import datetime

num_weeks: int  # number of weeks
players_per_group: int  # players per group
num_groups: int  # number of groups
num_players: int  # players per group * number of groups
time_budget = 600
show_additional_info = True
show_additional_info_str = "Yes"
id_variable: int

sat_solver: Solver

all_clauses = []
id_counter = 1

def generate_all_clauses():
    ensure_golfer_plays_exactly_once_per_week()
    ensure_group_contains_exactly_p_players()
    ensure_no_repeated_players_in_groups()
    symmetry_breaking1()
    symmetry_breaking2()

def plus_clause(clause):
    sat_solver.add_clause(clause)
    all_clauses.append(clause)

# (EO) Using sequential encounter
def exactly_one(var: list[int]):
    global id_variable
    n = len(var)
    assert n == num_groups

    # Create new variables S1, S2, ..., Sn-1
    S = [0]
    for i in range(1, n):
        id_variable += 1
        S.append(id_variable)

    # Add clauses to the SAT solver
    # X1 -> S1
    plus_clause([-var[0], S[1]])

    for i in range(2, n):
        # (Xi v Si-1) -> Si
        # <=> (¬Xi ^ ¬Si-1) v Si
        # <=> (Si v ¬Xi) ^ (Si v ¬Si-1)
        plus_clause([S[i], -var[i - 1]])
        plus_clause([S[i], -S[i - 1]])
        # Si-1 -> ¬Xi
        plus_clause([-S[i - 1], -var[i]])

    # Sn-1 -> ¬Xn
    plus_clause([-S[n - 1], -var[n - 1]])

    # ALO
    clause = []
    for i in range(0, n):
        clause.append(var[i])
    plus_clause(clause)


# (EO) Using binomial
# def exactly_one(var: list[int]):
#     n = var.__len__()
#     assert n == num_groups

#     # (1): (ALO)
#     clause = []
#     for i in range(0, n):
#         clause.append(var[i])
#     plus_clause(clause)

#     # (2): (AMO)
#     for i in range (0, n):
#         for j in range (i + 1, n):
#             plus_clause([-1 * var[i], -1 * var[j]])

# Every golfer plays exactly once a week
# x_w_g
def ensure_golfer_plays_exactly_once_per_week():
    for player in range(1, num_players + 1):
        for week in range(1, num_weeks + 1):
            list = []
            for group in range(1, num_groups + 1):
                list.append(get_variable(player, group, week))
            exactly_one(list)

# (EK) Using new sequential encounter
def exactly_k(var: list[int], k):
    global id_variable
    n = var.__len__() - 1
    assert n == num_players
    map_register = [[0 for j in range(k + 1)] for i in range(n)]
    for i in range(1, n):
        for j in range(1, min(i, k) + 1):
            id_variable += 1
            map_register[i][j] = id_variable

    # (1): If a bit is true, the first bit of the corresponding register is true
    for i in range(1, n):
        plus_clause([-1 * var[i], map_register[i][1]])

    # (2): R[i - 1][j] = 1, R[i][j] = 1;
    for i in range(2, n):
        for j in range(1, min(i - 1, k) + 1):
            plus_clause([-1 * map_register[i - 1][j], map_register[i][j]])

    # (3): If bit i is on and R[i - 1][j - 1] = 1, R[i][j] = 1;
    for i in range(2, n):
        for j in range(2, min(i, k) + 1):
            plus_clause([-1 * var[i], -1 * map_register[i - 1][j - 1], map_register[i][j]])

    # (4): If bit i is off and R[i - 1][j] = 0, R[i][j] = 0;
    for i in range(2, n):
        for j in range(1, min(i - 1, k) + 1):
            plus_clause([var[i], map_register[i - 1][j], -1 * map_register[i][j]])

    # (5): If bit i is off, R[i][i] = 0;
    for i in range(1, k + 1):
        plus_clause([var[i], -1 * map_register[i][i]])

    # (6): If R[i - 1][j - 1] = 0, R[i][j] = 0;
    for i in range(2, n):
        for j in range(2, min(i, k) + 1):
            plus_clause([map_register[i - 1][j - 1], -1 * map_register[i][j]])

    # (7): (At least k) R[n - 1][k] = 1 or (n-th bit is true and R[n - 1][k - 1] = 1)
    plus_clause([map_register[n - 1][k], var[n]])
    plus_clause([map_register[n - 1][k], map_register[n - 1][k - 1]])

    # (8): (At most k) If i-th bit is true, R[i - 1][k] = 0;
    for i in range(k + 1, n + 1):
        plus_clause([-1 * var[i], -1 * map_register[i - 1][k]])

# A group contains exactly p players
# w_g_x
def ensure_group_contains_exactly_p_players():
    for week in range(1, num_weeks + 1):
        for group in range(1, num_groups + 1):
            list = [-1]
            for player in range(1, num_players + 1):
                list.append(get_variable(player, group, week))
            exactly_k(list, players_per_group)

# Ensures that no players are repeated in the same group across different weeks and groups.
# w_g_x_x_g_w (3)
def ensure_no_repeated_players_in_groups():
    for week in range(1, num_weeks + 1):
        for group in range(1, num_groups + 1):
            for golfer1 in range(1, num_players + 1):
                for golfer2 in range(golfer1 + 1, num_players + 1):
                    for other_group in range(1, num_groups + 1):
                        for other_week in range(week + 1, num_weeks + 1):
                            clause = [-1 * get_variable(golfer1, group, week),
                                      -1 * get_variable(golfer2, group, week),
                                      -1 * get_variable(golfer1, other_group, other_week),
                                      -1 * get_variable(golfer2, other_group, other_week)]
                            plus_clause(clause)

# Symmetry breaking 1: p người chơi đầu tiên được đặt vào nhóm đầu tiên của tuần đầu tiên; p người chơi tiếp theo, ở nhóm thứ hai của tuần đầu tiên; và cứ tiếp tục như vậy.
# $\bigwedge_{i=1}^x \mathrm{G}_{i [(i-1)/p + 1] 1}$
def symmetry_breaking1():
    for i in range(1, players_per_group + 1):
        clause = [get_variable(i, (i - 1) // players_per_group + 1, 1)]
        plus_clause(clause)
        
# Symmetry breaking 2: p người chơi đầu tiên được đặt vào nhóm đầu tiên của tuần đầu tiên; p người chơi tiếp theo, ở nhóm thứ hai của tuần đầu tiên; và cứ tiếp tục như vậy.
# $\bigwedge_{l=2}^w \bigwedge_{k=1}^p \mathrm{G}_{k k l}$
def symmetry_breaking2():
    for l in range(2, num_weeks + 1):
        for k in range(1, players_per_group + 1):
            clause = [get_variable(k, k, l)]
            plus_clause(clause)


# returns a unique identifier for the variable that represents the assignment of the golfer to the group in the week
def get_variable(golfer, group, week):
    golfer -= 1
    group -= 1
    week -= 1
    return 1 + golfer + (group * num_players) + (week * num_players * num_groups);

def resolve_variable(v):
    for golfer in range(1, num_players + 1):
        for week in range(1, num_weeks + 1):
            for group in range(1, num_groups + 1):
                if abs(v) == get_variable(golfer, group, week):
                    return golfer, group, week

def process_results(results):
    new_table = {}
    for week in range(1, num_weeks + 1):
        new_table[week] = {}
        for group in range(1, num_groups + 1):
            new_table[week][group] = []
    for row in results:
        new_table[row["week"]][row["group"]].append(row["golfer"])
    return new_table

def show_results(results):
    print_to_console_and_log_table = PrettyTable()
    field_names = ["Week"]
    for group in range(1, num_groups + 1):
        field_names.append("Group " + str(group))
    print_to_console_and_log_table.field_names = field_names
    for week in range(1, num_weeks + 1):
        row = [str(week)]
        for group in range(1, num_groups + 1):
            row.append(str(",".join(list(map(str, results[week][group])))))
        print_to_console_and_log_table.add_row(row)
    print_to_console_and_log(print_to_console_and_log_table)

def interrupt(s):
    s.interrupt()

# solve the problem using the SAT Solver and write the results to xlsx file
def solve_sat_problem():
    global num_players, id_variable, sat_solver
    num_players = players_per_group * num_groups
    id_variable = num_players * num_groups * num_weeks

    # Clear the all_clauses list
    all_clauses.clear()

    print_to_console_and_log(f"\nGenerating a problem.\nNumber of groups: {num_groups}.\nPlayers per group: {players_per_group}.\nNumber of weeks: {num_weeks}.")

    sat_solver = Glucose3(use_timer=True)
    generate_all_clauses()

    # Store the number of variables and clauses before solving the problem
    num_vars = sat_solver.nof_vars()
    num_clauses = sat_solver.nof_clauses()

    if show_additional_info:
        print_to_console_and_log("Clauses: " + str(sat_solver.nof_clauses()))
        print_to_console_and_log("Variables: " + str(sat_solver.nof_vars()))

    # Create the directory if it doesn't exist
    directory_path = "input_cnf"
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)

    # Create the full path to the file "{problem}.cnf" in the directory "input_cnf"
    problem_name = f"{num_groups}-{players_per_group}-{num_weeks}"
    file_name = problem_name + ".cnf"
    file_path = os.path.join(directory_path, file_name)

    # Write data to the file
    with open(file_path, 'w') as writer:
        # Write a line of information about the number of variables and constraints
        writer.write("p cnf " + str(num_vars) + " " + str(num_clauses) + "\n")

        # Write each clause to the file
        for clause in all_clauses:
            for literal in clause:
                writer.write(str(literal) + " ")
            writer.write("0\n")

    print_to_console_and_log("\nCNF written to " + file_path + ".\n")

    print_to_console_and_log("Searching for a solution.")

    timer = Timer(time_budget, interrupt, [sat_solver])
    timer.start()

    start_time = time.time()
    sat_status = sat_solver.solve_limited(expect_interrupt=True)

    global id_counter

    result_dict = {
        "ID": id_counter,
        "Problem": f"{num_groups}-{players_per_group}-{num_weeks}-new_sc_sb",
        "Type": "new_sc_sb",
        "Time": "",
        "Result": "",
        "Variables": 0,
        "Clauses": 0
    }

    id_counter += 1

    if sat_status is False:
        end_time = time.time()
        elapsed_time = end_time - start_time
        print_to_console_and_log("UNSAT. Time run: (" + '{0:.3f}s'.format(elapsed_time) + ").\n")
        result_dict["Result"] = "unsat"
        result_dict["Time"] = '{0:.3f}'.format(elapsed_time)
        result_dict["Variables"] = sat_solver.nof_vars()
        result_dict["Clauses"] = sat_solver.nof_clauses()
    else:
        solution = sat_solver.get_model()
        if solution is None:
            end_time = time.time()
            elapsed_time = end_time - start_time
            if elapsed_time > time_budget:
                print_to_console_and_log("Time limit exceeded (" + '{0:.3f}s'.format(elapsed_time) + ").\n")
                result_dict["Result"] = "timeout"
                result_dict["Time"] = "timeout"
            else:
                print_to_console_and_log("What da hell. Time run: (" + '{0:.3f}s'.format(elapsed_time) + ").\n")
                result_dict["Result"] = "unsat"
                result_dict["Time"] = '{0:.3f}'.format(elapsed_time)
            result_dict["Variables"] = sat_solver.nof_vars()
            result_dict["Clauses"] = sat_solver.nof_clauses()
        else:
            print_to_console_and_log(
                "A solution was found in time " + '{0:.3f}s'.format(sat_solver.time()) + ". Generating it now.\n")
            result_dict["Result"] = "sat"

            results = []
            for v in solution:
                if v > 0 and v <= num_players * num_groups * num_weeks:
                    ikl = resolve_variable(v)
                    golfer, group, week = ikl
                    results.append({"golfer": golfer, "group": group, "week": week})

            final_result = process_results(results)
            show_results(final_result)

            if show_additional_info:
                sat_accum_stats = sat_solver.accum_stats()
                print_to_console_and_log("Restarts: " +
                        str(sat_accum_stats['restarts']) +
                        ", decisions: " +
                        str(sat_accum_stats['decisions']) +
                        ", propagations: " +
                        str(sat_accum_stats["propagations"]))

            result_dict["Time"] = '{0:.3f}'.format(sat_solver.time())
            result_dict["Variables"] = sat_solver.nof_vars()
            result_dict["Clauses"] = sat_solver.nof_clauses()

            sat_solver.delete()

    # Append the result to a list
    excel_results = []
    excel_results.append(result_dict)

    # Write the results to an Excel file
    if not os.path.exists("drive/MyDrive/out/NewSCSB"):
    # Create the 'drive/out' folder
      os.makedirs("drive/MyDrive/out/NewSCSB")

    df = pd.DataFrame(excel_results)
    current_date = datetime.now().strftime('%Y-%m-%d')
    excel_file_path = f"drive/MyDrive/out/NewSCSB/results_{current_date}.xlsx"

    # Check if the file already exists
    if os.path.exists(excel_file_path):
        try:
            book = load_workbook(excel_file_path)
        except BadZipFile:
            book = Workbook()  # Create a new workbook if the file is not a valid Excel file

        # Check if the 'Results' sheet exists
        if 'Results' not in book.sheetnames:
            book.create_sheet('Results')  # Create 'Results' sheet if it doesn't exist

        sheet = book['Results']

        for row in dataframe_to_rows(df, index=False, header=False):
            sheet.append(row)

        book.save(excel_file_path)

    else:
        df.to_excel(excel_file_path, index=False, sheet_name='Results', header=False)

    print_to_console_and_log("Result written to Excel file:", os.path.abspath(excel_file_path))  # Print_print_to_console_and_log full path
    print_to_console_and_log("Result added to Excel file.\n")

# Open the log file in append mode

log_file = open("drive/MyDrive/out/NewSCSB/output.txt", "a")

# Define a custom print_to_console_and_log function that writes to both console and log file
def print_to_console_and_log(*args, **kwargs):
    print(*args, **kwargs)
    print(*args, file=log_file, **kwargs)
    log_file.flush()

# read input data from file data.txt (many lines, each line is number of weeks, number of players per group, number of groups)
# solve the problem
def run_from_input_file():
    global num_groups, players_per_group, num_weeks
    with open("drive/MyDrive/data.txt") as f:
        for line in f:
            num_groups, players_per_group, num_weeks = map(int, line.split())
            assert num_groups > 1 and players_per_group > 1
            solve_sat_problem()

if __name__ == "__main__":
    # main_menu()
    run_from_input_file()


Generating a problem.
Number of groups: 3.
Players per group: 2.
Number of weeks: 2.
Clauses: 393
Variables: 90

CNF written to input_cnf/3-2-2.cnf.

Searching for a solution.
A solution was found in time 0.000s. Generating it now.

+------+---------+---------+---------+
| Week | Group 1 | Group 2 | Group 3 |
+------+---------+---------+---------+
|  1   |   2,5   |   1,3   |   4,6   |
|  2   |   1,2   |   3,4   |   5,6   |
+------+---------+---------+---------+
Restarts: 1, decisions: 12, propagations: 105
Result written to Excel file: /content/drive/MyDrive/out/NewSC/results_2024-04-24.xlsx
Result added to Excel file.


Generating a problem.
Number of groups: 5.
Players per group: 3.
Number of weeks: 2.
Clauses: 4485
Variables: 540

CNF written to input_cnf/5-3-2.cnf.

Searching for a solution.
A solution was found in time 0.000s. Generating it now.

+------+---------+---------+---------+----------+----------+
| Week | Group 1 | Group 2 | Group 3 | Group 4  | Group 5  |
+------+----